In [1]:
import os
import cv2
import numpy as np
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt

# Define constants
data_path = "/Users/yanthraa/Desktop/PROJECT/NEW DATASET/archive"
target_size = (224, 224)
target_count = 1000  # Desired number of images per class
output_path = "/Users/yanthraa/Desktop/PROJECT/NEW DATASET/Augmented_Dataset"  # Output directory

# Ensure output directories exist
os.makedirs(output_path, exist_ok=True)
for class_name in ['Negatives', 'Positives']:
    os.makedirs(os.path.join(output_path, class_name), exist_ok=True)

# Define augmentation sequence
augmenters = iaa.Sequential([
    iaa.Fliplr(0.5),  # Horizontal flip
    iaa.Affine(rotate=(-15, 15)),  # Rotate between -15 and 15 degrees
    iaa.GaussianBlur(sigma=(0.0, 1.0)),  # Apply Gaussian blur
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add noise
    iaa.Multiply((0.8, 1.2))  # Brightness adjustments
])

# Load and preprocess data
images = []
labels = []

for class_name in ['Negatives', 'Positives']:
    class_path = os.path.join(data_path, class_name)
    class_images = []
    label = 0 if class_name == 'Negatives' else 1
    
    # Load original images
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, target_size)
        img = img.astype(np.float32) / 255.0
        class_images.append(img)
    
    # Augment images until target count is reached
    while len(class_images) < target_count:
        augmented_images = augmenters(images=class_images[:min(len(class_images), target_count - len(class_images))])
        class_images.extend(augmented_images)
    
    class_images = class_images[:target_count]  # Trim excess
    images.extend(class_images)
    labels.extend([label] * target_count)
    
    # Save augmented images
    for idx, img in enumerate(class_images):
        save_path = os.path.join(output_path, class_name, f"aug_{idx}.png")
        cv2.imwrite(save_path, (img * 255).astype(np.uint8))

# Convert to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Print class proportions
print("Class Proportions:")
print("Negatives:", np.sum(labels == 0), "Positives:", np.sum(labels == 1))


Class Proportions:
Negatives: 1000 Positives: 1000


In [2]:
from sklearn.model_selection import train_test_split

# Separate data for Negatives and Positives classes
Negatives_images = images[labels == 0][:500]
Positives_images = images[labels == 1][:500]
Negatives_labels = labels[labels == 0][:500]
Positives_labels = labels[labels == 1][:500]

# Concatenate the data back together
balanced_images = np.concatenate([Negatives_images, Positives_images])
balanced_labels = np.concatenate([Negatives_labels, Positives_labels])


# Split the data into training and testing sets
X_train_full, X_test, y_train_full, y_test = train_test_split(balanced_images, balanced_labels, test_size=0.2, random_state=42)

# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.1, random_state=42)

#print the balanced data
print("Balanced Images:",balanced_images.shape)
print("Balanced labels:",balanced_labels.shape)

#print the data seperately of each class
print("Negatives_images:",Negatives_images.shape)
print("Positives_images:",Positives_images.shape)


# Print the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


Balanced Images: (1000, 224, 224)
Balanced labels: (1000,)
Negatives_images: (500, 224, 224)
Positives_images: (500, 224, 224)
X_train shape: (720, 224, 224)
X_test shape: (200, 224, 224)
y_train shape: (720,)
y_test shape: (200,)


In [ ]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# from tensorflow.keras.optimizers import Optimizer
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# from sklearn.model_selection import train_test_split, KFold
# from sklearn.metrics import (confusion_matrix, classification_report, roc_curve, 
#                            roc_auc_score, precision_recall_curve, auc, precision_score, 
#                            recall_score, f1_score, cohen_kappa_score, matthews_corrcoef)
# import matplotlib.pyplot as plt
# import seaborn as sns
# import time
# import os

# # Create directory for results if it doesn't exist
# os.makedirs('results', exist_ok=True)

# # Custom Optimizers
# import tensorflow as tf
# from tensorflow.keras.optimizers import Optimizer
# import tensorflow.keras.backend as K
# class HGSOptimizer(tf.keras.optimizers.Optimizer):
#     def __init__(self, 
#                  learning_rate=0.001,
#                  population_size=30,
#                  hunger_rate=0.1,
#                  name="HGSOptimizer",
#                  **kwargs):
#         super().__init__(name=name, learning_rate=learning_rate, **kwargs)
#         self._population_size = population_size
#         self._hunger_rate = hunger_rate
    
#     def update_step(self, gradient, variable, learning_rate):
#         """Update step for variables."""
#         hunger_factor = tf.random.uniform([], dtype=variable.dtype) * self._hunger_rate
#         survival_factor = 1.0 - hunger_factor
        
#         # Apply the update
#         variable.assign_sub(learning_rate * gradient * survival_factor)

# class MPAOptimizer(tf.keras.optimizers.Optimizer):
#     def __init__(self,
#                  learning_rate=0.001,
#                  elite_factor=0.2,
#                  prey_factor=0.1,
#                  name="MPAOptimizer",
#                  **kwargs):
#         super().__init__(name=name, learning_rate=learning_rate, **kwargs)
#         self._elite_factor = elite_factor
#         self._prey_factor = prey_factor
    
#     def update_step(self, gradient, variable, learning_rate):
#         """Update step for variables."""
#         elite_influence = tf.random.uniform([], dtype=variable.dtype) * self._elite_factor
#         prey_influence = tf.random.uniform([], dtype=variable.dtype) * self._prey_factor
        
#         # Apply the update
#         variable.assign_sub(learning_rate * gradient * (1 + elite_influence - prey_influence))

# class PathwaysOptimizer(tf.keras.optimizers.Optimizer):
#     def __init__(self,
#                  learning_rate=0.001,
#                  pathway_strength=0.3,
#                  adaptation_rate=0.1,
#                  name="PathwaysOptimizer",
#                  **kwargs):
#         super().__init__(name=name, learning_rate=learning_rate, **kwargs)
#         self._pathway_strength = pathway_strength
#         self._adaptation_rate = adaptation_rate
    
#     def update_step(self, gradient, variable, learning_rate):
#         """Update step for variables."""
#         pathway_factor = tf.random.uniform([], dtype=variable.dtype) * self._pathway_strength
#         adapt_factor = tf.random.uniform([], dtype=variable.dtype) * self._adaptation_rate
        
#         # Apply the update
#         variable.assign_sub(learning_rate * gradient * (1 + pathway_factor * adapt_factor))

#     def get_config(self):
#         config = super().get_config()
#         config.update({
#             "pathway_strength": self._pathway_strength,
#             "adaptation_rate": self._adaptation_rate
#         })
#         return config

# def create_model(optimizer, input_shape=(224, 224, 3)):
#     model = Sequential([
#         Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
#         MaxPooling2D((2, 2)),
#         Conv2D(64, (3, 3), activation='relu', padding='same'),
#         MaxPooling2D((2, 2)),
#         Conv2D(64, (3, 3), activation='relu', padding='same'),
#         MaxPooling2D((2, 2)),
#         Flatten(),
#         Dense(128, activation='relu'),
#         Dropout(0.5),
#         Dense(1, activation='sigmoid')
#     ])
    
#     model.compile(
#         optimizer=optimizer,
#         loss='binary_crossentropy',
#         metrics=['accuracy', 
#                 tf.keras.metrics.Precision(),
#                 tf.keras.metrics.Recall(),
#                 tf.keras.metrics.AUC()]
#     )
#     return model

# def save_metrics_to_csv(metrics_dict, optimizer_name, fold, k):
#     """Save metrics for a single run to CSV"""
#     df = pd.DataFrame({
#         'Test Accuracy': [metrics_dict['Test Accuracy']],
#         'Train Accuracy': [metrics_dict['Train Accuracy']],
#         'Precision': [metrics_dict['Precision']],
#         'Recall': [metrics_dict['Recall']],
#         'AUC-ROC': [metrics_dict['AUC-ROC']],
#         'AUC-PR': [metrics_dict['AUC-PR']],
#         'TN': [metrics_dict['TN']],
#         'FP': [metrics_dict['FP']],
#         'FN': [metrics_dict['FN']],
#         'TP': [metrics_dict['TP']],
#         'F1 Score': [metrics_dict['F1 Score']],
#         'Cohen\'s Kappa Coefficient': [metrics_dict['Cohen\'s Kappa']],
#         'Matthews Correlation Coefficient': [metrics_dict['Matthews Correlation Coefficient']],
#         'Training Loss': [metrics_dict['Training Loss']],
#         'Testing Loss': [metrics_dict['Testing Loss']],
#         'Time Taken (seconds)': [metrics_dict['Time Taken (seconds)']],
#         'Fold': [fold],
#         'K': [k]
#     })
    
#     filename = f'results/{optimizer_name}_metrics_K{k}_Fold{fold}.csv'
#     df.to_csv(filename, index=False)
#     return filename

# def plot_training_curves(history, optimizer_name, k, fold):
#     """Plot and save training curves"""
#     plt.figure(figsize=(12, 4))
    
#     # Accuracy plot
#     plt.subplot(1, 2, 1)
#     plt.plot(history.history['accuracy'], label='train')
#     plt.plot(history.history['val_accuracy'], label='validation')
#     plt.title(f'{optimizer_name} - Accuracy (K={k}, Fold={fold})')
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy')
#     plt.legend()
    
#     # Loss plot
#     plt.subplot(1, 2, 2)
#     plt.plot(history.history['loss'], label='train')
#     plt.plot(history.history['val_loss'], label='validation')
#     plt.title(f'{optimizer_name} - Loss (K={k}, Fold={fold})')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.legend()
    
#     plt.tight_layout()
#     plt.savefig(f'results/{optimizer_name}_training_curves_K{k}_Fold{fold}.png')
#     plt.close()

# def plot_comparative_metrics(all_summary_dfs):
#     """Create comparative visualizations for all optimizers"""
#     plt.figure(figsize=(15, 10))
    
#     metrics_to_plot = ['Test Accuracy', 'F1 Score', 'AUC-ROC', 'Training Loss']
#     for i, metric in enumerate(metrics_to_plot, 1):
#         plt.subplot(2, 2, i)
#         data = pd.concat(all_summary_dfs)
#         sns.boxplot(data=data, x='K', y=metric, hue='Optimizer')
#         plt.title(f'{metric} by K-Fold and Optimizer')
#         plt.xticks(rotation=45)
    
#     plt.tight_layout()
#     plt.savefig('results/comparative_metrics.png')
#     plt.close()


# def preprocess_data(X_train, X_test, y_train, y_test):
#     """Preprocess the data to ensure correct shapes and normalization"""
#     # Add channel dimension if not present
#     if len(X_train.shape) == 3:
#         X_train = X_train[..., np.newaxis]
#     if len(X_test.shape) == 3:
#         X_test = X_test[..., np.newaxis]
    
#     # Convert to RGB by repeating the channel 3 times
#     X_train = np.repeat(X_train, 3, axis=-1)
#     X_test = np.repeat(X_test, 3, axis=-1)
    
#     # Normalize pixel values
#     X_train = X_train.astype('float32') / 255.0
#     X_test = X_test.astype('float32') / 255.0
    
#     # Convert labels to numpy arrays if they aren't already
#     y_train = np.array(y_train)
#     y_test = np.array(y_test)
    
#     print("Preprocessed shapes:")
#     print(f"X_train: {X_train.shape}")
#     print(f"X_test: {X_test.shape}")
#     print(f"y_train: {y_train.shape}")
#     print(f"y_test: {y_test.shape}")
    
#     return X_train, X_test, y_train, y_test

# def create_model(optimizer, input_shape):
#     """Create model with specified input shape"""
#     model = Sequential([
#         Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
#         MaxPooling2D((2, 2)),
#         Conv2D(64, (3, 3), activation='relu', padding='same'),
#         MaxPooling2D((2, 2)),
#         Conv2D(64, (3, 3), activation='relu', padding='same'),
#         MaxPooling2D((2, 2)),
#         Flatten(),
#         Dense(128, activation='relu'),
#         Dropout(0.5),
#         Dense(1, activation='sigmoid')
#     ])
    
#     model.compile(
#         optimizer=optimizer,
#         loss='binary_crossentropy',
#         metrics=['accuracy', 
#                 tf.keras.metrics.Precision(),
#                 tf.keras.metrics.Recall(),
#                 tf.keras.metrics.AUC()]
#     )
#     return model
# def train_and_evaluate_single_optimizer(X_train, y_train, X_test, y_test, optimizer_name, k_folds=[5, 10, 15, 20]):
#     """Train and evaluate a single optimizer with detailed metrics saving"""
    
#     all_files = []
#     summary_metrics = []
    
#     # Ensure input shape is correct
#     input_shape = X_train.shape[1:]  # Should be (224, 224, 3)
#     print(f"Model input shape: {input_shape}")
    
#     for k in k_folds:
#         print(f'\nTraining with k={k} folds')
#         kfold = KFold(n_splits=k, shuffle=True, random_state=42)
        
#         for fold, (train_ids, val_ids) in enumerate(kfold.split(X_train)):
#             print(f'Training fold {fold + 1}/{k}')
            
#             try:
#                 # Split data
#                 X_train_fold = X_train[train_ids]
#                 y_train_fold = y_train[train_ids]
#                 X_val_fold = X_train[val_ids]
#                 y_val_fold = y_train[val_ids]
                
#                 print(f"Fold data shapes:")
#                 print(f"X_train_fold: {X_train_fold.shape}")
#                 print(f"y_train_fold: {y_train_fold.shape}")
#                 print(f"X_val_fold: {X_val_fold.shape}")
#                 print(f"y_val_fold: {y_val_fold.shape}")
                
#                 # Create optimizer
#                 if optimizer_name == "HGS":
#                     optimizer = HGSOptimizer(learning_rate=0.001, hunger_rate=0.1)
#                 elif optimizer_name == "MPA":
#                     optimizer = MPAOptimizer(learning_rate=0.001, elite_factor=0.2)
#                 else:  # Pathways
#                     optimizer = PathwaysOptimizer(learning_rate=0.001, pathway_strength=0.3)
                
#                 # Create model with correct input shape
#                 model = create_model(optimizer, input_shape)
                
#                 early_stopping = EarlyStopping(
#                     monitor='val_loss',
#                     patience=5,
#                     restore_best_weights=True
#                 )
                
#                 checkpoint = ModelCheckpoint(
#                     f'results/{optimizer_name}_model_K{k}_Fold{fold}.h5',
#                     monitor='val_loss',
#                     save_best_only=True
#                 )
                
#                 # Start timing
#                 start_time = time.time()
                
#                 # Train the model
#                 history = model.fit(
#                     X_train_fold, y_train_fold,
#                     epochs=50,
#                     batch_size=32,
#                     validation_data=(X_val_fold, y_val_fold),
#                     callbacks=[early_stopping, checkpoint],
#                     verbose=1
#                 )
                
#                 # End timing
#                 time_taken = time.time() - start_time
                
#                 # Evaluate model
#                 test_results = model.evaluate(X_test, y_test, verbose=0)
#                 y_pred = model.predict(X_test)
#                 y_pred_binary = (y_pred > 0.5).astype(int)
                
#                 # Calculate confusion matrix metrics
#                 tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
                
#                 # Calculate precision-recall curve for AUC-PR
#                 precision, recall, _ = precision_recall_curve(y_test, y_pred)
#                 pr_auc = auc(recall, precision)
                
#                 # Create metrics dictionary
#                 metrics = {
#                     'Test Accuracy': test_results[1],
#                     'Train Accuracy': history.history['accuracy'][-1],
#                     'Precision': precision_score(y_test, y_pred_binary),
#                     'Recall': recall_score(y_test, y_pred_binary),
#                     'AUC-ROC': roc_auc_score(y_test, y_pred),
#                     'AUC-PR': pr_auc,
#                     'TN': tn,
#                     'FP': fp,
#                     'FN': fn,
#                     'TP': tp,
#                     'F1 Score': f1_score(y_test, y_pred_binary),
#                     'Cohen\'s Kappa': cohen_kappa_score(y_test, y_pred_binary),
#                     'Matthews Correlation Coefficient': matthews_corrcoef(y_test, y_pred_binary),
#                     'Training Loss': history.history['loss'][-1],
#                     'Testing Loss': test_results[0],
#                     'Time Taken (seconds)': time_taken
#                 }
                
#                 # Save metrics to CSV
#                 csv_file = save_metrics_to_csv(metrics, optimizer_name, fold + 1, k)
#                 all_files.append(csv_file)
                
#                 # Plot and save training curves
#                 plot_training_curves(history, optimizer_name, k, fold + 1)
                
#                 # Store summary metrics
#                 summary_metrics.append({
#                     'Optimizer': optimizer_name,
#                     'K': k,
#                     'Fold': fold + 1,
#                     **metrics
#                 })
                
#                 print(f"\nFold {fold + 1} Results:")
#                 print(f"Test Accuracy: {metrics['Test Accuracy']:.4f}")
#                 print(f"F1 Score: {metrics['F1 Score']:.4f}")
#                 print(f"AUC-ROC: {metrics['AUC-ROC']:.4f}")
#                 print(f"Training Time: {metrics['Time Taken (seconds)']:.2f} seconds")
                
#             except Exception as e:
#                 print(f"Error in fold {fold + 1}: {str(e)}")
#                 continue
    
#     # Create and save summary DataFrame for this optimizer
#     if summary_metrics:
#         summary_df = pd.DataFrame(summary_metrics)
#         summary_df.to_csv(f'results/{optimizer_name}_summary_metrics.csv', index=False)
        
#         # Print overall results for this optimizer
#         print(f"\nOverall Results for {optimizer_name}:")
#         print(f"Average Test Accuracy: {summary_df['Test Accuracy'].mean():.4f} ± {summary_df['Test Accuracy'].std():.4f}")
#         print(f"Average F1 Score: {summary_df['F1 Score'].mean():.4f} ± {summary_df['F1 Score'].std():.4f}")
#         print(f"Average AUC-ROC: {summary_df['AUC-ROC'].mean():.4f} ± {summary_df['AUC-ROC'].std():.4f}")
#         print(f"Average Training Time: {summary_df['Time Taken (seconds)'].mean():.2f} ± {summary_df['Time Taken (seconds)'].std():.2f} seconds")
#     else:
#         summary_df = pd.DataFrame()  # Empty DataFrame if no metrics were collected
    
#     return summary_df, all_files

# # Main execution
# if __name__ == "__main__":
#     # First preprocess the data
#     X_train, X_test, y_train, y_test = preprocess_data(X_train, X_test, y_train, y_test)
    
#     # Define optimizers to test
#     optimizers = ["HGS", "MPA", "Pathways"]
#     all_summary_dfs = []
#     all_generated_files = []
    
#     # Train and evaluate each optimizer
#     for opt in optimizers:
#         print(f"\nStarting training with {opt} optimizer")
#         try:
#             summary_df, files = train_and_evaluate_single_optimizer(
#                 X_train, y_train, X_test, y_test, opt
#             )
#             all_summary_dfs.append(summary_df)
#             all_generated_files.extend(files)
#         except Exception as e:
#             print(f"Error training {opt} optimizer: {str(e)}")
#             continue
    
#     # Create comparative visualizations if we have results
#     if all_summary_dfs:
#         plot_comparative_metrics(all_summary_dfs)
    
#     # Print summary of generated files
#     print("\nGenerated Files:")
#     for file in all_generated_files:
#         print(f"- {file}")
        



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Optimizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (confusion_matrix, classification_report, roc_curve, 
                           roc_auc_score, precision_recall_curve, auc, precision_score, 
                           recall_score, f1_score, cohen_kappa_score, matthews_corrcoef)
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os

# Create directory for results if it doesn't exist
os.makedirs('results', exist_ok=True)

# Custom Optimizers
import tensorflow as tf
from tensorflow.keras.optimizers import Optimizer
import tensorflow.keras.backend as K
class HGSOptimizer(tf.keras.optimizers.Optimizer):
    def __init__(self, 
                 learning_rate=0.001,
                 population_size=30,
                 hunger_rate=0.1,
                 name="HGSOptimizer",
                 **kwargs):
        super().__init__(name=name, learning_rate=learning_rate, **kwargs)
        self._population_size = population_size
        self._hunger_rate = hunger_rate
    
    def update_step(self, gradient, variable, learning_rate):
        """Update step for variables."""
        hunger_factor = tf.random.uniform([], dtype=variable.dtype) * self._hunger_rate
        survival_factor = 1.0 - hunger_factor
        
        # Apply the update
        variable.assign_sub(learning_rate * gradient * survival_factor)


def create_model(optimizer, input_shape=(224, 224, 3)):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', 
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                tf.keras.metrics.AUC()]
    )
    return model

def save_metrics_to_csv(metrics_dict, optimizer_name, fold, k):
    """Save metrics for a single run to CSV"""
    df = pd.DataFrame({
        'Test Accuracy': [metrics_dict['Test Accuracy']],
        'Train Accuracy': [metrics_dict['Train Accuracy']],
        'Precision': [metrics_dict['Precision']],
        'Recall': [metrics_dict['Recall']],
        'AUC-ROC': [metrics_dict['AUC-ROC']],
        'AUC-PR': [metrics_dict['AUC-PR']],
        'TN': [metrics_dict['TN']],
        'FP': [metrics_dict['FP']],
        'FN': [metrics_dict['FN']],
        'TP': [metrics_dict['TP']],
        'F1 Score': [metrics_dict['F1 Score']],
        'Cohen\'s Kappa Coefficient': [metrics_dict['Cohen\'s Kappa']],
        'Matthews Correlation Coefficient': [metrics_dict['Matthews Correlation Coefficient']],
        'Training Loss': [metrics_dict['Training Loss']],
        'Testing Loss': [metrics_dict['Testing Loss']],
        'Time Taken (seconds)': [metrics_dict['Time Taken (seconds)']],
        'Fold': [fold],
        'K': [k]
    })
    
    filename = f'results/{optimizer_name}_metrics_K{k}_Fold{fold}.csv'
    df.to_csv(filename, index=False)
    return filename

def plot_training_curves(history, optimizer_name, k, fold):
    """Plot and save training curves"""
    plt.figure(figsize=(12, 4))
    
    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='validation')
    plt.title(f'{optimizer_name} - Accuracy (K={k}, Fold={fold})')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.title(f'{optimizer_name} - Loss (K={k}, Fold={fold})')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f'results/{optimizer_name}_training_curves_K{k}_Fold{fold}.png')
    plt.close()

def plot_comparative_metrics(all_summary_dfs):
    """Create comparative visualizations for all optimizers"""
    plt.figure(figsize=(15, 10))
    
    metrics_to_plot = ['Test Accuracy', 'F1 Score', 'AUC-ROC', 'Training Loss']
    for i, metric in enumerate(metrics_to_plot, 1):
        plt.subplot(2, 2, i)
        data = pd.concat(all_summary_dfs)
        sns.boxplot(data=data, x='K', y=metric, hue='Optimizer')
        plt.title(f'{metric} by K-Fold and Optimizer')
        plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig('results/comparative_metrics.png')
    plt.close()


def preprocess_data(X_train, X_test, y_train, y_test):
    """Preprocess the data to ensure correct shapes and normalization"""
    # Add channel dimension if not present
    if len(X_train.shape) == 3:
        X_train = X_train[..., np.newaxis]
    if len(X_test.shape) == 3:
        X_test = X_test[..., np.newaxis]
    
    # Convert to RGB by repeating the channel 3 times
    X_train = np.repeat(X_train, 3, axis=-1)
    X_test = np.repeat(X_test, 3, axis=-1)
    
    # Normalize pixel values
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    
    # Convert labels to numpy arrays if they aren't already
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    print("Preprocessed shapes:")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    print(f"y_train: {y_train.shape}")
    print(f"y_test: {y_test.shape}")
    
    return X_train, X_test, y_train, y_test

def create_model(optimizer, input_shape):
    """Create model with specified input shape"""
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', 
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                tf.keras.metrics.AUC()]
    )
    return model
def train_and_evaluate_single_optimizer(X_train, y_train, X_test, y_test, optimizer_name, k_folds=[5, 10, 15, 20]):
    """Train and evaluate a single optimizer with detailed metrics saving"""
    
    all_files = []
    summary_metrics = []
    
    # Ensure input shape is correct
    input_shape = X_train.shape[1:]  # Should be (224, 224, 3)
    print(f"Model input shape: {input_shape}")
    
    for k in k_folds:
        print(f'\nTraining with k={k} folds')
        kfold = KFold(n_splits=k, shuffle=True, random_state=42)
        
        for fold, (train_ids, val_ids) in enumerate(kfold.split(X_train)):
            print(f'Training fold {fold + 1}/{k}')
            
            try:
                # Split data
                X_train_fold = X_train[train_ids]
                y_train_fold = y_train[train_ids]
                X_val_fold = X_train[val_ids]
                y_val_fold = y_train[val_ids]
                
                print(f"Fold data shapes:")
                print(f"X_train_fold: {X_train_fold.shape}")
                print(f"y_train_fold: {y_train_fold.shape}")
                print(f"X_val_fold: {X_val_fold.shape}")
                print(f"y_val_fold: {y_val_fold.shape}")
                
                # Create optimizer
                if optimizer_name == "HGS":
                    optimizer = HGSOptimizer(learning_rate=0.001, hunger_rate=0.1)
                elif optimizer_name == "MPA":
                    optimizer = MPAOptimizer(learning_rate=0.001, elite_factor=0.2)
                else:  # Pathways
                    optimizer = PathwaysOptimizer(learning_rate=0.001, pathway_strength=0.3)
                
                # Create model with correct input shape
                model = create_model(optimizer, input_shape)
                
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True
                )
                
                checkpoint = ModelCheckpoint(
                    f'results/{optimizer_name}_model_K{k}_Fold{fold}.h5',
                    monitor='val_loss',
                    save_best_only=True
                )
                
                # Start timing
                start_time = time.time()
                
                # Train the model
                history = model.fit(
                    X_train_fold, y_train_fold,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val_fold, y_val_fold),
                    callbacks=[early_stopping, checkpoint],
                    verbose=1
                )
                
                # End timing
                time_taken = time.time() - start_time
                
                # Evaluate model
                test_results = model.evaluate(X_test, y_test, verbose=0)
                y_pred = model.predict(X_test)
                y_pred_binary = (y_pred > 0.5).astype(int)
                
                # Calculate confusion matrix metrics
                tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
                
                # Calculate precision-recall curve for AUC-PR
                precision, recall, _ = precision_recall_curve(y_test, y_pred)
                pr_auc = auc(recall, precision)
                
                # Create metrics dictionary
                metrics = {
                    'Test Accuracy': test_results[1],
                    'Train Accuracy': history.history['accuracy'][-1],
                    'Precision': precision_score(y_test, y_pred_binary),
                    'Recall': recall_score(y_test, y_pred_binary),
                    'AUC-ROC': roc_auc_score(y_test, y_pred),
                    'AUC-PR': pr_auc,
                    'TN': tn,
                    'FP': fp,
                    'FN': fn,
                    'TP': tp,
                    'F1 Score': f1_score(y_test, y_pred_binary),
                    'Cohen\'s Kappa': cohen_kappa_score(y_test, y_pred_binary),
                    'Matthews Correlation Coefficient': matthews_corrcoef(y_test, y_pred_binary),
                    'Training Loss': history.history['loss'][-1],
                    'Testing Loss': test_results[0],
                    'Time Taken (seconds)': time_taken
                }
                
                # Save metrics to CSV
                csv_file = save_metrics_to_csv(metrics, optimizer_name, fold + 1, k)
                all_files.append(csv_file)
                
                # Plot and save training curves
                plot_training_curves(history, optimizer_name, k, fold + 1)
                
                # Store summary metrics
                summary_metrics.append({
                    'Optimizer': optimizer_name,
                    'K': k,
                    'Fold': fold + 1,
                    **metrics
                })
                
                print(f"\nFold {fold + 1} Results:")
                print(f"Test Accuracy: {metrics['Test Accuracy']:.4f}")
                print(f"F1 Score: {metrics['F1 Score']:.4f}")
                print(f"AUC-ROC: {metrics['AUC-ROC']:.4f}")
                print(f"Training Time: {metrics['Time Taken (seconds)']:.2f} seconds")
                
            except Exception as e:
                print(f"Error in fold {fold + 1}: {str(e)}")
                continue
    
    # Create and save summary DataFrame for this optimizer
    if summary_metrics:
        summary_df = pd.DataFrame(summary_metrics)
        summary_df.to_csv(f'results/{optimizer_name}_summary_metrics.csv', index=False)
        
        # Print overall results for this optimizer
        print(f"\nOverall Results for {optimizer_name}:")
        print(f"Average Test Accuracy: {summary_df['Test Accuracy'].mean():.4f} ± {summary_df['Test Accuracy'].std():.4f}")
        print(f"Average F1 Score: {summary_df['F1 Score'].mean():.4f} ± {summary_df['F1 Score'].std():.4f}")
        print(f"Average AUC-ROC: {summary_df['AUC-ROC'].mean():.4f} ± {summary_df['AUC-ROC'].std():.4f}")
        print(f"Average Training Time: {summary_df['Time Taken (seconds)'].mean():.2f} ± {summary_df['Time Taken (seconds)'].std():.2f} seconds")
    else:
        summary_df = pd.DataFrame()  # Empty DataFrame if no metrics were collected
    
    return summary_df, all_files

# Main execution
if __name__ == "__main__":
    # First preprocess the data
    X_train, X_test, y_train, y_test = preprocess_data(X_train, X_test, y_train, y_test)
    
    # Define optimizers to test
    optimizers = ["HGS", "MPA", "Pathways"]
    all_summary_dfs = []
    all_generated_files = []
    
    # Train and evaluate each optimizer
    for opt in optimizers:
        print(f"\nStarting training with {opt} optimizer")
        try:
            summary_df, files = train_and_evaluate_single_optimizer(
                X_train, y_train, X_test, y_test, opt
            )
            all_summary_dfs.append(summary_df)
            all_generated_files.extend(files)
        except Exception as e:
            print(f"Error training {opt} optimizer: {str(e)}")
            continue
    
    # Create comparative visualizations if we have results
    if all_summary_dfs:
        plot_comparative_metrics(all_summary_dfs)
    
    # Print summary of generated files
    print("\nGenerated Files:")
    for file in all_generated_files:
        print(f"- {file}")
        



In [10]:
# Extract results for HGS
# hgs_summary_df = all_summary_dfs[0]  # HGS should be the first optimizer
hgs_generated_files = all_generated_files[:len(files)]  # Files generated by HGS

# Display summary
print("HGS Summary:")
# print(hgs_summary_df)

# List the generated files
print("\nHGS Generated Files:")
for file in hgs_generated_files:
    print(f"- {file}")


HGS Summary:

HGS Generated Files:
- results/HGS_metrics_K5_Fold1.csv
- results/HGS_metrics_K5_Fold2.csv
- results/HGS_metrics_K5_Fold3.csv
- results/HGS_metrics_K5_Fold4.csv
- results/HGS_metrics_K5_Fold5.csv
- results/HGS_metrics_K10_Fold1.csv
- results/HGS_metrics_K10_Fold2.csv
- results/HGS_metrics_K10_Fold3.csv
- results/HGS_metrics_K10_Fold4.csv
- results/HGS_metrics_K10_Fold5.csv
- results/HGS_metrics_K10_Fold6.csv
- results/HGS_metrics_K10_Fold7.csv
- results/HGS_metrics_K10_Fold8.csv
- results/HGS_metrics_K10_Fold9.csv
- results/HGS_metrics_K10_Fold10.csv
- results/HGS_metrics_K15_Fold1.csv
- results/HGS_metrics_K15_Fold2.csv
- results/HGS_metrics_K15_Fold3.csv
- results/HGS_metrics_K15_Fold4.csv
- results/HGS_metrics_K15_Fold5.csv
- results/HGS_metrics_K15_Fold6.csv
- results/HGS_metrics_K15_Fold7.csv
- results/HGS_metrics_K15_Fold8.csv
- results/HGS_metrics_K15_Fold9.csv
- results/HGS_metrics_K15_Fold10.csv
- results/HGS_metrics_K15_Fold11.csv
- results/HGS_metrics_K15_Fold1

In [12]:
import pandas as pd
import glob

# Path to all HGS result files
file_paths = sorted(glob.glob("results/HGS_metrics_*.csv"))  # Fetch all matching CSV files

# Read and concatenate all files
df_list = [pd.read_csv(file) for file in file_paths]
merged_df = pd.concat(df_list, ignore_index=True)

# Save to a single CSV
merged_df.to_csv("results/HGS_merged_results.csv", index=False)

print("Merged CSV saved as: results/HGS_merged_results.csv")


Merged CSV saved as: results/HGS_merged_results.csv


Lion Optimization Algorithm (LOA)

Salp Swarm Algorithm (SSA)

Grey Wolf Optimizer (GWO)

AdamW

Arithmetic Optimization Algorithm (AOA)

Sharpness-Aware Minimization (SAM)

Stochastic Weight Averaging (SWA)

LAMB (Layer-wise Adaptive Moments optimizer for Batch training)

Sine Cosine Algorithm (SCA) with Neural Extension Chaos Optimization Algorithms (COA


Hunger Games Search (HGS)

Marine Predators Algorithm (MPA)

Pathways Optimizer (2024)


Hunger Games Search (HGS):


Inspired by the survival competition concept
Key features for image classification:

Adaptive population size helps find optimal features
Balance between exploration (searching new areas) and exploitation (refining existing solutions)
Hunger mechanism helps escape local optima


Significance for your data:

Good for handling high-dimensional image data (224x224x3 = 150,528 dimensions)
Can help find discriminative features between your binary classes
The hunger mechanism can help prevent overfitting




Marine Predators Algorithm (MPA):


Based on marine predator foraging strategies
Key features:

Three phases of optimization: cruise, hunt, and chase
Elite-based updating mechanism
Adaptive movement patterns


Significance for your data:

Multi-phase search can help find subtle differences between classes
Elite preservation helps maintain good features
Particularly good at handling noisy data
Adaptive step sizes can help with fine-grained feature learning




Pathways Optimizer (2024):


Inspired by neural pathway formation
Key features:

Dynamic pathway strength adaptation
Parallel path exploration
Learning rate adjustment based on pathway success


Significance for your data:

Well-suited for deep learning architectures
Can adapt to different feature importances
Good at handling the hierarchical nature of image features



